#### Importing all requisite packages

In [1]:
import pandas as pd
import datetime

#### Read the data file using pandas

In [2]:
flights = pd.read_csv('flights.txt', sep='|')

#### Analyze the data
Since the data file is around 300 MB in size, it can be analyzed in my local machine. If the file was big (in GBs or TBs), we would have used Apache Spark for data analysis and transformation. </br>
We can see below that the data file has 1191805 rows with 31 columns

In [3]:
flights.shape

(1191805, 31)

We can see below that the DataFrame has 31 columns. The Identifier column seems to be **TRANSACTIONID**. The next steps wuld be to check the following
- TRANSACTIONID: See if it has all unique values
- FLIGHTDATE: Can we convert it to date type
- ORIGINSTATE and ORIGINSTATENAME have a few missing values. See what is missing and if it can be replaced. These two can be used as filters in reporting
- DESTSTATE and DESTSTATENAME also have few missing values
- CRSDEPTIME, DEPTIME, CRSARRTIME, ARRTIME - Check data and see if it can be converted to datetime.time object
- CANCELLED, DIVERTED: Check data and see if they can be converted to boolean type
- DISTANCE: Can it be converted to a numeric data type

In [4]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1191805 entries, 0 to 1191804
Data columns (total 31 columns):
TRANSACTIONID        1191805 non-null int64
FLIGHTDATE           1191805 non-null int64
AIRLINECODE          1191805 non-null object
AIRLINENAME          1191805 non-null object
TAILNUM              1034988 non-null object
FLIGHTNUM            1191805 non-null int64
ORIGINAIRPORTCODE    1191805 non-null object
ORIGAIRPORTNAME      1191805 non-null object
ORIGINCITYNAME       1191805 non-null object
ORIGINSTATE          1180963 non-null object
ORIGINSTATENAME      1180963 non-null object
DESTAIRPORTCODE      1191805 non-null object
DESTAIRPORTNAME      1191805 non-null object
DESTCITYNAME         1191805 non-null object
DESTSTATE            1180967 non-null object
DESTSTATENAME        1180967 non-null object
CRSDEPTIME           1191805 non-null int64
DEPTIME              1163470 non-null float64
DEPDELAY             1163470 non-null float64
TAXIOUT              1011833 non-n

### Datatype conversions

### **1. TRANSACTIONID** has all unique values

In [5]:
flights['TRANSACTIONID'].unique().size

1191805

### 2. Converting **FLIGHTDATE** to datetime

In [6]:
flights['FLIGHTDATE'] = pd.to_datetime(flights.FLIGHTDATE, format="%Y%m%d")
flights['FLIGHTDATE'].head()

0   2002-01-01
1   2002-01-01
2   2002-01-01
3   2002-01-01
4   2002-01-03
Name: FLIGHTDATE, dtype: datetime64[ns]

### 3. Dealing with null values in ORIGINSTATE and ORIGINSTATENAME
As we can see below, we have 10482 null values for ORIGINSTATE and ORIGINSTATENAME. Origin State can be used as a filter in reports and would be a good-to-have. Since 10482 is a large enough value, we can see if we can deduce it from within the data or if we can provide it from external sources.

In [7]:
flights.ORIGINSTATE.value_counts(dropna=False)

CA     141852
TX     137592
IL      78413
FL      78338
GA      67182
NY      51653
PA      39624
CO      39422
NC      39249
AZ      39024
VA      37271
MO      35668
MI      33342
NV      31670
MN      27582
NJ      24711
TN      23971
WA      23216
OH      22753
MA      22310
UT      20364
KY      19137
MD      17515
LA      13271
OR      12143
HI      11125
NaN     10842
IN       9176
WI       8389
AK       8111
NM       7178
AL       7011
SC       5978
CT       5655
PR       4788
AR       4763
NE       4513
RI       3910
IA       3500
ID       3146
MT       3088
MS       2441
NH       2081
ND       1929
ME       1741
SD       1593
WY       1085
VT        913
VI        799
WV        654
TT        118
DE          5
Name: ORIGINSTATE, dtype: int64

<br/>
From below, we can confirm that we cannot deduce the missing State names from within the data. We will have to use external data sources to fill the data manually.

In [8]:
# Get all city names where State name is null
arrCityNamesWhereStateIsNull = flights[flights.ORIGINSTATE.isna()]['ORIGINCITYNAME'].unique()

# Filter data on above array to see if some city names hve corresponding State. 
flights[flights.ORIGINCITYNAME.isin(arrCityNamesWhereStateIsNull)]['ORIGINSTATE'].unique()

array([nan], dtype=object)

#### Printing the city names with no State Code and State Name

In [9]:
arrCityNamesWhereStateIsNull

array(['Lawton/Fort Sill', 'Manhattan/Ft. Riley', 'Hays', 'Oklahoma City',
       'Topeka', 'Tulsa', 'Wichita', 'Garden City'], dtype=object)

In [10]:
# Manually assigning the state code and state name for the missing city names
stateNames = {
    "Lawton/Fort Sill": ["Oklahoma", "OK"],
    "Manhattan/Ft. Riley": ["Kansas", "KS"],
    "Hays": ["Kansas", "KS"],
    "Oklahoma City": ["Oklahoma", "OK"],
    "Topeka": ["Kansas", "KS"],
    "Tulsa" :["Oklahoma", "OK"],
    "Wichita": ["Kansas", "KS"],
    "Garden City": ["Kansas", "KS"]
}

####
## Function to update the Origin State Code and State Name for city names passed as parameters
####
def updateOriginState(cityName, stateCode, stateName):
    flights.loc[flights.ORIGINCITYNAME == cityName, 'ORIGINSTATE'] = stateCode
    flights.loc[flights.ORIGINCITYNAME == cityName, 'ORIGINSTATENAME'] = stateName

In [11]:
for cityName in arrCityNamesWhereStateIsNull:
    updateOriginState(cityName, stateNames.get(cityName)[1], stateNames.get(cityName)[0])

flights.ORIGINSTATE.unique()

array(['NM', 'GA', 'NY', 'TX', 'OH', 'AL', 'MT', 'OR', 'AK', 'LA', 'WA',
       'MN', 'ME', 'MD', 'IL', 'CA', 'NJ', 'WI', 'NC', 'MI', 'PA', 'CO',
       'WV', 'WY', 'UT', 'SC', 'IA', 'VT', 'PR', 'SD', 'TN', 'MO', 'MA',
       'ND', 'ID', 'FL', 'NV', 'HI', 'TT', 'KY', 'OK', 'AR', 'NE', 'MS',
       'IN', 'NH', 'VA', 'KS', 'CT', 'AZ', 'DE', 'RI', 'VI'], dtype=object)

### 4. Dealing with null values in DESTSTATE and DESTSTATENAME
As we can see below, we have 10838 null values for DESTSTATE and DESTSTATENAME. Origin State can be used as a filter in reports and would be a good-to-have. Since 10838 is a large enough value, we can see if we can deduce it from within the data or if we can provide it from external sources.

In [12]:
flights.DESTSTATE.value_counts(dropna=False)

CA     141630
TX     137460
IL      78897
FL      78405
GA      66713
NY      51876
PA      39728
CO      39644
NC      39348
AZ      38955
VA      37050
MO      35714
MI      33584
NV      31506
MN      27530
NJ      24847
TN      23961
WA      23223
OH      22818
MA      22144
UT      20257
KY      19056
MD      17429
LA      13429
OR      12086
HI      11110
NaN     10838
IN       9161
WI       8239
AK       8074
NM       7081
AL       6975
SC       6078
CT       5671
PR       4781
AR       4771
NE       4480
RI       3918
IA       3469
ID       3239
MT       3095
MS       2516
NH       2130
ND       1921
ME       1802
SD       1603
WY       1053
VT        876
VI        808
WV        677
TT        132
DE         17
Name: DESTSTATE, dtype: int64

In [13]:
# Get all city names where State name is null
destCityNamesWhereStateIsNull = flights[flights.DESTSTATE.isna()]['DESTCITYNAME'].unique()
destCityNamesWhereStateIsNull

array(['Oklahoma City', 'Tulsa', 'Wichita', 'Lawton/Fort Sill',
       'Manhattan/Ft. Riley', 'Topeka', 'Garden City', 'Hays'],
      dtype=object)

In [14]:
####
## Function to update the Destination State Code and State Name for city names passed as parameters
####
def updateDestState(cityName, stateCode, stateName):
    flights.loc[flights.DESTCITYNAME == cityName, 'DESTSTATE'] = stateCode
    flights.loc[flights.DESTCITYNAME == cityName, 'DESTSTATENAME'] = stateName

In [15]:
for cityName in arrCityNamesWhereStateIsNull:
    updateDestState(cityName, stateNames.get(cityName)[1], stateNames.get(cityName)[0])

flights.DESTSTATE.unique()

array(['TX', 'MO', 'NV', 'KY', 'CA', 'AZ', 'IL', 'CO', 'FL', 'MN', 'WA',
       'GA', 'MD', 'NJ', 'UT', 'OR', 'PA', 'OH', 'VA', 'NY', 'MI', 'NC',
       'WI', 'ME', 'MA', 'TN', 'SC', 'VI', 'OK', 'PR', 'IA', 'IN', 'AL',
       'LA', 'MS', 'AR', 'NM', 'CT', 'WV', 'KS', 'NE', 'RI', 'NH', 'HI',
       'MT', 'AK', 'DE', 'SD', 'WY', 'VT', 'ND', 'ID', 'TT'], dtype=object)

### 5. Converting time columns to PostgreSQL time data type (HH24:MI:SS format)
CRSDEPTIME (Scheduled Departure Time) and CRSARRTIME (Scheduled Arrival Time) do not have null values in the data, so they can be converted to 00:00:00 time format. DEPTIME and ARRTIME have NaN values, so they are float by default.

In [16]:
arrScheduledTimeCols = ['CRSDEPTIME', 'CRSARRTIME']

for timeCol in arrScheduledTimeCols:
    flights[timeCol] = flights[timeCol].map(lambda x:str(x).zfill(4)).map(lambda x:x[:2] + ':' + x[2:] + ':00')
#    flights[timeCol] = pd.to_timedelta(flights[timeCol], unit="s")

In [17]:
arrActualTimeCols = ['DEPTIME', 'ARRTIME']

####
## Function to left pad the string values with 0. Ignore null values.
####
def padZero(x):
    if x == 'nan':
        return None
    else:
        # Remove the .0 t the end and left pad with 0
        return x[:-2].zfill(4)

def convertToTime(x):
    if x == None:
        return x
    else:
        return x[:2] + ':' + x[2:] + ':00'

for timeCol in arrActualTimeCols:
    flights[timeCol] = flights[timeCol].astype('str').map(padZero).map(convertToTime)

### 6. Converting CANCELLED and DIVERTED to boolean
The columns CANCELLED and DIVERTED have the following values - T, True, 1, F, False, 0. The following code replaces T, True and 1 to numpy boolean type True and F, False and 1 to numpy boolean type True.

In [18]:
flights['CANCELLED'].replace({'F': False, 'False': False, '0': False}, inplace=True)
flights['CANCELLED'].replace({'T': True, 'True': True, '1': True}, inplace=True)
flights['CANCELLED'] = flights['CANCELLED'].astype('bool')
flights['CANCELLED'].unique()

array([False,  True])

In [19]:
flights['DIVERTED'].replace({'F': False, 'False': False, '0': False}, inplace=True)
flights['DIVERTED'].replace({'T': True, 'True': True, '1': True}, inplace=True)
flights['DIVERTED'] = flights['DIVERTED'].astype('bool')
flights['DIVERTED'].unique()

array([False,  True])

### 7. Converting DISTANCE to float
The column DISTANCE if of the folowing format - X miles. We need to first check whether every value is followed by the word 'miles'. If every value is followed by miles, we can remove it and convert the column to float.

In [20]:
flights['DISTANCE'].str.split(expand=True)[1].unique()

array(['miles'], dtype=object)

In [21]:
flights['DISTANCE'] = flights['DISTANCE'].str.split(expand=True)[0]
flights['DISTANCE'] = flights['DISTANCE'].astype('float')

### 8. Adding DISTANCE_GROUP column

In [22]:
flights['DISTANCE_GROUP'] = \
(flights['DISTANCE'].astype(int).floordiv(100)*100).map(str) + \
'-' + \
(flights['DISTANCE'].astype(int).floordiv(100)*100+100).map(str)

### 9. Adding a column named DEPDELAY_GT15 indicating if the departure delay is greater than 15 minutes

In [23]:
flights['DEPDELAY_GT15'] = flights['DEPDELAY'] > 15

### 10. Removing Airline Code from name

In [24]:
flights['AIRLINENAME'] = flights['AIRLINENAME'].map(lambda x: x.split(':')[0])

### 11. Clean up the Airport Name by removing the concatenated city and state

In [25]:
flights['ORIGAIRPORTNAME'] = flights['ORIGAIRPORTNAME'].map(lambda x: x.split(':')[1].strip())
flights['DESTAIRPORTNAME'] = flights['DESTAIRPORTNAME'].map(lambda x: x.split(':')[1].strip())

### 12. Converting DataFrame to parquet
Saving the DataFrame as a parquet format. The parquet file format is a binary file which stores the schema information as well. The file can later be read and stored to a database.

In [26]:
flights.to_parquet('flights.parquet')